In [ ]:
!pip install keybert
!pip install flair
!pip install keras-bert

     |████████████████████████████████| 78 kB 5.6 MB/s 
     |████████████████████████████████| 213 kB 20.3 MB/s 
     |████████████████████████████████| 51 kB 8.7 MB/s 
     |████████████████████████████████| 3.1 MB 69.8 MB/s 
     |████████████████████████████████| 3.3 MB 23.3 MB/s 
     |████████████████████████████████| 1.2 MB 68.6 MB/s 
     |████████████████████████████████| 59 kB 8.9 MB/s 
     |████████████████████████████████| 596 kB 83.2 MB/s 
     |████████████████████████████████| 895 kB 28.8 MB/s 
  Created wheel for keybert: filename=keybert-0.5.0-py3-none-any.whl size=20491 sha256=de114102bcded57ad4d05b14ef2389070a7f332493e0b4b95cbf0018469cc2b8
  Stored in directory: /root/.cache/pip/wheels/99/1f/3f/590d2997adbb2d0e1f82e8ee05d42d6910e92c3ed283015ff8
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=c4f7533f49eb23a4455389a2e743b6c06808e2a66b2b98e5b2c0767e10f4e15c
  Stored in directory: /root/.cache/pip/whee

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Nov 15 06:47:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
bert_path = '/gdrive/MyDrive/dacon/stack_bert/bert/uncased_L-12_H-768_A-12/'
file_path = '/gdrive/MyDrive/dacon/stack_bert/transfer-learning-on-stack-exchange-tags/data/'

In [ ]:
import tensorflow as tf

import pandas as pd
import numpy as np
import re
import pickle

import keras as keras
from keras.models import load_model
from keras import backend as K
from keras import Input, Model
from keras import optimizers

import codecs
from tqdm import tqdm
import shutil

In [ ]:
from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps

ModuleNotFoundError: ignored

In [ ]:
def copytree(src, dst, symlinks=False, ignore = None):
  for item in os.listdir(src):
    s = os.path.join(src, item)
    d = os.path.join(dst, item)
    if os.path.isdir(s):
      shutil.copytree(s, d, symlinks, ignore)
    else:
      shutil.copy2(s, d)

In [ ]:
import os
os.makedirs('bert')
# 존재하면 error 뜸

In [ ]:
copytree(os.path.join(bert_path), 'bert')

In [ ]:
biology = pd.read_csv(file_path+'biology.csv') #['id', 'title', 'content', 'tags']
cooking = pd.read_csv(file_path+'cooking.csv')
crypto = pd.read_csv(file_path+'crypto.csv')
diy = pd.read_csv(file_path+'diy.csv')
robotics = pd.read_csv(file_path+'robotics.csv')
test = pd.read_csv(file_path+'test.csv')
sample = pd.read_csv(file_path+'sample_submission.csv')

# data = {'biology':biology, 'cooking':cooking, 'crypto':crypto, 'diy':diy, 'robotics':robotics}
data = biology.append(cooking, ignore_index=True)
data = data.append(crypto, ignore_index=True)
data = data.append(diy, ignore_index=True)
data = data.append(robotics, ignore_index=True)
# data = pd.concat
# data = biology+cooking+crypto+diy+robotics
print(len(biology))
print(len(cooking))
print(len(crypto))
print(len(diy))
print(len(robotics))
print(len(test))
print(len(sample))

13196
15404
10432
25918
2771
81926
81926


In [ ]:
print(crypto)

          id  ...                                               tags
0          3  ...                       block-cipher des permutation
1          7  ...  oblivious-transfer multiparty-computation func...
2          8  ...                                         sha-1 hash
3          9  ...             hash cryptanalysis preimage-resistance
4         14  ...                          encryption rsa public-key
...      ...  ...                                                ...
10427  39758  ...                     hash hmac collision-resistance
10428  39759  ...                                       hash padding
10429  39761  ...                    perfect-secrecy forward-secrecy
10430  39762  ...                                hash diffie-hellman
10431  39768  ...                  encryption aes symmetric key-wrap

[10432 rows x 4 columns]


In [ ]:
from bs4 import BeautifulSoup as bs
from itertools import chain
def preprocess(data):
    # # data = re.sub(r'^\W+|\W+$','',data)
    # data = re.sub(r'\s',' ',data)
    # data = re.sub(r'\n',' ',data)
    removeHtmlTags = lambda x: bs(x, "html.parser").text
    data['cleaned_content'] = data.apply(removeHtmlTags)
    print("Removed html tags.")

    return data['cleaned_content']

def split_tag(tags):
    tag_list=[]
    for x in range(len(tags)):
        tag_list.append(str(tags[x]).split(' '))
    tag_list = list(chain.from_iterable(tag_list))
    uni_tag= np.unique(tag_list)
    print('uni=',len(uni_tag))
    dict_uni_tag = {string : i for i,string in enumerate(uni_tag)}
    print(dict_uni_tag)
    dict_mat_uni_tags=np.zeros((len(tags),len(uni_tag)))
    for c, i in enumerate(tags):
        for k, j in enumerate(i.split(' ')):
            dict_mat_uni_tags[c][dict_uni_tag[j]]=1
    return tag_list, uni_tag, dict_mat_uni_tags

In [ ]:
tag_list, uni_tag, dict_mat_uni_tags = split_tag(data.tags)
content = preprocess(data.content)
title = preprocess(data.title)
test_content = preprocess(test.content)
test_title = preprocess(test.title)


uni= 2668
{'120-240v': 0, '240v': 1, '2nd-preimage-resistance': 2, '3d-model': 3, '3d-printing': 4, '3d-reconstruction': 5, '3d-structure': 6, '3des': 7, 'abe': 8, 'abiogenesis': 9, 'abs': 10, 'accelerometer': 11, 'access-control': 12, 'access-panel': 13, 'accessibility': 14, 'accumulators': 15, 'acid': 16, 'acidity': 17, 'acoustic': 18, 'acoustic-cryptanalysis': 19, 'acoustic-rangefinder': 20, 'acrylic': 21, 'action-potential': 22, 'activerobot': 23, 'actuator': 24, 'adaptation': 25, 'addiction': 26, 'addition': 27, 'additives': 28, 'adhesive': 29, 'adversarial-model': 30, 'advice': 31, 'aes': 32, 'afci': 33, 'affine-cipher': 34, 'african': 35, 'aging': 36, 'agriculture': 37, 'aids': 38, 'air-conditioner': 39, 'air-conditioning': 40, 'air-filter': 41, 'air-leaks': 42, 'air-muscle': 43, 'air-quality': 44, 'alarm': 45, 'alcohol': 46, 'alcohol-content': 47, 'alfredo': 48, 'algebraic-attack': 49, 'algebraic-eraser': 50, 'algorithm': 51, 'algorithm-design': 52, 'alkalinity': 53, 'alkyd': 5

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Removed html tags.
Removed html tags.
Removed html tags.
Removed html tags.


In [ ]:
print(pd.DataFrame(tag_list).value_counts())
print(uni_tag)
print(len(uni_tag))

electrical       4490
plumbing         2223
encryption       1783
wiring           1680
human-biology    1448
                 ... 
shades              1
cloud               1
sip                 1
chime               1
rtfm                1
Length: 2668, dtype: int64
['120-240v' '240v' '2nd-preimage-resistance' ... 'zero-knowledge-proofs'
 'zoology' 'zucchini']
2668


In [ ]:
for i in data:
    t.tags.split
print(len(sample))

67721
81926


In [ ]:

all_data = content + title
test_data = test_content + test_title

In [ ]:
get_data = all_data[16001]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

n_gram_range = (1, 2)
stop_words = "english"

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([get_data])
all_candidates = count.get_feature_names()

In [ ]:
print(all_candidates[:10])

['actually', 'actually lack', 'bake', 'bake perplexed', 'board', 'board coating', 'cake', 'cake layers', 'cake pans', 'chalk']


In [ ]:
from keybert import KeyBERT
from flair.embeddings import TransformerDocumentEmbeddings

roberta = TransformerDocumentEmbeddings('roberta-base')
kw_model = KeyBERT(model=roberta)
nr_candidate = 50
# keywords =kw_model.extract_keywords(get_data, keyphrase_ngram_range=(1, 1), stop_words='english', 
                                    # use_maxsum=True, nr_candidates=20, top_n=5, diversity=0.7)
# keywords = kw_model.extract_keywords(get_data)
# keywords2 =kw_model.extract_keywords(get_data, keyphrase_ngram_range=(1, 2), stop_words='english', 
#                               use_maxsum=True, nr_candidates=nr_candidate, top_n=15, diversity=0.7)
keywords3 =kw_model.extract_keywords(get_data, keyphrase_ngram_range=(1, 2), stop_words='english', 
                                     nr_candidates=nr_candidate, top_n=30, diversity=0.7)
keywords4 =kw_model.extract_keywords(get_data, keyphrase_ngram_range=(1, 2), stop_words='english', 
                              use_mmr=True, nr_candidates=nr_candidate, top_n=30, diversity=0.7)


# keywords5 =kw_model.extract_keywords(get_data, keyphrase_ngram_range=(1,2), stop_words='english', 
#                               use_maxsum=True, nr_candidates=nr_candidate, top_n=5, diversity=0.7,seed_keywords=['immunology', 'cell-biology', 'hematology'])
# keywords6 =kw_model.extract_keywords(get_data, keyphrase_ngram_range=(1,2), stop_words='english', 
#                               use_maxsum=True, nr_candidates=nr_candidate, top_n=5, diversity=0.7,seed_keywords=[ 'cell-biology', 'hematology'])
# keywords7 =kw_model.extract_keywords(get_data, keyphrase_ngram_range=(1,2), stop_words='english', 
#                               use_maxsum=True, nr_candidates=nr_candidate, top_n=5, diversity=0.7,seed_keywords=[ 'hematology'])
# keywords8 =kw_model.extract_keywords(get_data, keyphrase_ngram_range=(1,2), stop_words='english', 
#                               use_maxsum=True, nr_candidates=nr_candidate, top_n=5, diversity=0.7,seed_keywords=[ 'immunology'])
# from keybert._mmr import mmr
# a = mmr(roberta, get_data,['immunology'])

# print(keywords)
# print(keywords2)
print(keywords3)
print(keywords4)
# print(keywords5)
# print(keywords6)
# print(keywords7)
# print(keywords8)
# print(a)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

[('wash', 0.9947), ('washed', 0.9947), ('washing', 0.9947), ('strange touch', 0.9946), ('like chalky', 0.9946), ('yellow cake', 0.9946), ('washer', 0.9946), ('didn', 0.9946), ('circular cake', 0.9946), ('comes', 0.9946), ('rub', 0.9946), ('feeling', 0.9945), ('hand washing', 0.9945), ('turned silvery', 0.9945), ('lazy', 0.9945), ('finally', 0.9945), ('board coating', 0.9945), ('scratch safe', 0.9945), ('turned', 0.9945), ('chalky white', 0.9945), ('instead', 0.9945), ('white coating', 0.9945), ('coating', 0.9945), ('destroyed', 0.9945), ('washed cake', 0.9945), ('feeling lazy', 0.9945), ('bake', 0.9945), ('lack coating', 0.9945), ('actually', 0.9945), ('ve', 0.9945)]
[('wash', 0.9947), ('bake perplexed', 0.9944), ('uneven pans', 0.9932), ('instead hand', 0.9929), ('turned silvery', 0.9945), ('like chalky', 0.9946), ('scratch safe', 0.9945), ('coating guessing', 0.994), ('fingers rub', 0.9944), ('pans comes', 0.994), ('realized didn', 0.994), ('lazy tossed', 0.9935), ('guessing actually

In [ ]:
import re
def list_value(data):
    value=[]
    for i in data:
        if len(i)==1:
            value.append(i[0])
            continue
        value.append(i[0])
        value.append(i[1])
    return value
keywords = [ re.sub(r' ','-',candidate[0]) for candidate in keywords4]
# word = list_value(keywords)
print(keywords)
# print(word)

['wash', 'bake-perplexed', 'uneven-pans', 'instead-hand', 'turned-silvery', 'like-chalky', 'scratch-safe', 'coating-guessing', 'fingers-rub', 'pans-comes', 'realized-didn', 'lazy-tossed', 'guessing-actually', 'looking-like', 'washed-cake', 'destroyed-uneven', 'touch-like', 'know-safe', 'layers-feeling', 'home-yellow', 'safe-bake', 'don-know', 'washing-realized', 'actually-lack', 'got-circular', 'rub-scratch', 'machine-washed', 'finally-use', 'like-chalk', 'cake-layers']


In [ ]:
print(sample)
print(test)

           id            tags
0           1  physics poetry
1           2  physics poetry
2           3  physics poetry
3           7  physics poetry
4           9  physics poetry
...       ...             ...
81921  278119  physics poetry
81922  278120  physics poetry
81923  278121  physics poetry
81924  278124  physics poetry
81925  278126  physics poetry

[81926 rows x 2 columns]
           id  ...                                            content
0           1  ...  <p>I often hear about subatomic particles havi...
1           2  ...  <p>How would you explain string theory to non ...
2           3  ...  <p>This is a question that has been posted at ...
3           7  ...  <p>What are the main problems that we need to ...
4           9  ...  <p>Hamilton's principle states that a dynamic ...
...       ...  ...                                                ...
81921  278119  ...  <p>A projectile is fired with a speed 6.25 m/s...
81922  278120  ...  <p>I can understand the generation

In [ ]:
#결과 예측
from keybert import KeyBERT
from flair.embeddings import TransformerDocumentEmbeddings
def write_file(file_name, data):
    data = re.sub(r',',' ',str(data))
    data = re.sub(r'\[','',str(data))
    data = re.sub(r'\]','',str(data))
    data = re.sub(r'\'',' ',str(data))
    with open(file_path+file_name, 'a') as file:
        file.write(str(data)+'\n')
roberta = TransformerDocumentEmbeddings('roberta-base')
kw_model = KeyBERT(model=roberta)
result=[]
j=77001 #23001 77001
print(test_data.shape)
for i , value in tqdm(enumerate(test_data.iloc[77746:])): #23135it [8:25:33,  1.07it/s], 54611it [19:46:12,  1.95s/it]
    if i % 1000 == 0:
        j = i + 77000
    keywords =kw_model.extract_keywords(value, keyphrase_ngram_range=(1, 2), stop_words='english', 
                                use_mmr=True, nr_candidates=nr_candidate, top_n=20, diversity=0.7)
    keyword = [re.sub(r' ','-',candidate[0]) for candidate in keywords]
    write_file(f'reset_result/result_{j}.txt', keyword)
    # result.append(keyword)

(81926,)


4180it [1:05:22,  1.07it/s]


In [ ]:
import os
# import chain
from itertools import chain
file_list_txt=[]
file_list = os.listdir(file_path+'reset/')
file_list_txt.append([file for file in file_list if file.endswith(".txt")])
# file_list = os.listdir(file_path+'reset_result/')
# file_list_txt.append([file for file in file_list if file.endswith(".txt")])
file_list_txt = list(chain.from_iterable(file_list_txt))
print(file_list_txt[0][7:-4])
# file_list_txt = map(sorted,file_list_txt[:][7:-4])
sort_file_list_txt =[]
for i in file_list_txt:
    sort_file_list_txt.append(int(i[7:-4]))
print(sort_file_list_txt)
sort_file_list_txt = sorted(sort_file_list_txt)
print ("file_list_txt: {}".format(sort_file_list_txt))
print(len(sort_file_list_txt))

78000
[78000, 77000, 79000, 81000, 80000, 45000, 44000, 42000, 43000, 41000, 4000, 39000, 40000, 35000, 33000, 34000, 32000, 3000, 31000, 30000, 37000, 38000, 36000, 28000, 29000, 25000, 26000, 23000, 24000, 22000, 27000, 21000, 2000, 20000, 19000, 18000, 17000, 14000, 15000, 16000, 13000, 12000, 23001, 10000, 11000, 0, 1000, 8000, 7000, 6000, 9000, 51000, 53000, 52000, 50000, 5000, 54000, 48000, 49000, 46000, 47000]
file_list_txt: [0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000, 21000, 22000, 23000, 23001, 24000, 25000, 26000, 27000, 28000, 29000, 30000, 31000, 32000, 33000, 34000, 35000, 36000, 37000, 38000, 39000, 40000, 41000, 42000, 43000, 44000, 45000, 46000, 47000, 48000, 49000, 50000, 51000, 52000, 53000, 54000, 77000, 78000, 79000, 80000, 81000]
61


In [ ]:
sample = pd.read_csv(file_path+'sample_submission.csv')
print(sample.shape)
row = 0
c=0
d=0
x=0
import re
def nlp(data):
    result = []
    for i in data:
        result.append(re.sub(r'\n',' ',str(i)))
    return str(result)[1:-1] 
for i in sort_file_list_txt:
    print(i)
    with open(file_path+f'reset/result_{i}.txt') as txt:
        lines = txt.readlines()
        if len(lines) == 1001:
            c+=1
        x += len(lines)
    # print('len =',len(lines))
    if i == 23000:
        # print(f'{row}:{row+len(lines)-1}')
        sample.iloc[row:row+len(lines)-1,1] = lines[:-1]
        d+=len(lines[:-1])
        # print(len(lines))
        row += len(lines)-1
        continue
    if i == 54000 or i == 81000:
        # print(f'{row}:{row+len(lines)-1}')
        sample.iloc[row:row+len(lines),1] = lines
        d+=len(lines)
        # print(len(lines))
        row += len(lines)
        continue
    if i == 23001 or i == 77000:
        # print(f'{row}:{row+len(lines)}')
        sample.iloc[row:row+len(lines),1] = lines
        d+=len(lines)
        row += len(lines)
        continue
    # print(f'{row}:{row+1000}')
    sample.iloc[row:row+1000,1] = lines[:1000]
    d+=len(lines[:1000])
    row += 1000
    # print('nomal')
# print('row = ',row)
    print('c =',c)
    print('d =',d)
    print('x=',x)
print(sample)
sample.to_csv(file_path+'keybert_test.csv',index=False)
       

In [ ]:
data = pd.read_csv(file_path+"sub_freq2.csv")
data.to_csv(file_path+'sub_sort2.csv',index=False)

In [ ]:
with open(file_path+'result0.txt','r') as f:
    print(f.readline())
    print(f.readline())

 spinning-motion    like-think    spin-does    axis-like    spin-mean    actually-relate    relates-subatomic    relate-spinning    think-particles    doesn-actually    does-spin    like    having-property    hear-subatomic    mean-actual    particles-having    particles-spin    property-called    spin-doesn    subatomic-particles 

 theory-non    physicists-specially    specially-interested    plausible-needed    explain-string    simplest-explanation    non-physicists    needed-successfully    prove-simplest    interested-plausible    non    explanation    successfully-prove    string-theory    plausible    specially    explanation-string    theory    physicists    explain 



In [ ]:
result1 = re.sub(r',',' ',str(result))
print(result1)
result1 = re.sub(r'\[','',str(result1))
result1 = re.sub(r'\]','',str(result1))
result1 = re.sub(r'\'',' ',str(result1))
sample1.tags = result
print(sample)
print(sample.tags[:8])

'spinning-motion'  'like-think'  'spin-does'  'axis-like'  'spin-mean'  'actually-relate'  'relates-subatomic'  'relate-spinning'  'think-particles'  'doesn-actually'  'does-spin'  'like'  'having-property'  'hear-subatomic'  'mean-actual'  'particles-having'  'particles-spin'  'property-called'  'spin-doesn'  'subatomic-particles'  'spin-relates'  'mean'  'actual-spinning'  'motion-spin'  'property'  'relates'  'called-spin'  'particles'  'having'  'spinning-axis'    'specifically-thinking'  'physicists-care'  'start-clarify'  'answer-seen'  'acting-certain'  'care-certain'  'posted-different'  'groups-think'  'action-maybe'  'things-invariant'  'relation-standard'  'different-forums'  'better-conceptual'  'means-sort'  'sort-group'  'question-posted'  'space-lie'  'good-start'  'thinking-symmetry'  'think-representation'  'acting-lie'  'acting-acting'  'representations-particle'  'action'  'think-relation'  'conceptual-answer'  'clarify-specifically'  'groups-people'  'model-care'  '

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
sample.to_csv(file_path+'result.csv')

In [ ]:
print(pd.read_csv(file_path+'result.csv'))

       Unnamed: 0      id                                               tags
0               0       1   spinning-motion    like-think    spin-does   ...
1               1       2   spinning-motion    like-think    spin-does   ...
2               2       3   spinning-motion    like-think    spin-does   ...
3               3       7   spinning-motion    like-think    spin-does   ...
4               4       9   spinning-motion    like-think    spin-does   ...
...           ...     ...                                                ...
81921       81921  278119                                     physics poetry
81922       81922  278120                                     physics poetry
81923       81923  278121                                     physics poetry
81924       81924  278124                                     physics poetry
81925       81925  278126                                     physics poetry

[81926 rows x 3 columns]
